In [1]:
import pandas as pd
import sqlite3 
import pytz
import datetime
import logging
import os 
import plotly.express as px 
import numpy as np 
import sys 
tz = pytz.timezone('UTC')
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", 200)
pd.options.mode.chained_assignment = None  # default='warn'
%load_ext autoreload
%autoreload 2

In [2]:
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(asctime)s :: %(levelname)s :: %(message)s')


In [3]:
def get_cnts_pcts(series):
    cnts = series.value_counts()
    pcts = 100*series.value_counts(normalize=True).round(5)
    combo = pd.concat({"Count":cnts, "Percent": pcts}, axis=1)
    return(combo)


In [4]:
def summarize_dog_pop(df):
    
    out = {}
    out['Count'] = df.shape[0]
    
    def parse_cnts_pcts1(df, desc):
        try:
            dict1 = {f'Top {desc} Name': df.iloc[0].name, 
                     f'Top {desc} %' : df.iloc[0]['Percent'], 
                    }
        except IndexError:
            dict1 = {f'Top {desc} Name': None, 
                     f'Top {desc} %' : None, 
                    }
        return(dict1)
    
    def parse_cnts_pcts2(df, desc):
        try:
            dict1 = {f'2nd {desc} Name': df.iloc[1].name, 
                     f'2nd {desc} %' : df.iloc[1]['Percent'], 
                }
        except IndexError:
            dict1 = {f'2nd {desc} Name': None, 
                     f'2nd {desc} %' : None, 
                }
        return(dict1)
    
    breeds = get_cnts_pcts(df['breeds.primary'])
    breeds_dict1 = parse_cnts_pcts1(breeds, 'Breed')
    breeds_dict2 = parse_cnts_pcts2(breeds, 'Breed')

    age = get_cnts_pcts(df['age'])
    age_dict1 = parse_cnts_pcts1(age, 'Age')
    age_dict2 = parse_cnts_pcts2(age, 'Age')
    
    color = get_cnts_pcts(df['colors.primary'])
    color_dict1 = parse_cnts_pcts1(color, 'Color')
    color_dict2 = parse_cnts_pcts2(color, 'Color')
    
    
    children = get_cnts_pcts(df['environment.children'])
    children_dict1 = parse_cnts_pcts1(children, 'Children')
    children_dict2 = parse_cnts_pcts2(children, 'Children')
    
    housetrained = get_cnts_pcts(df['attributes.house_trained'])
    housetrained_dict1 = parse_cnts_pcts1(housetrained, 'House Trained')
    housetrained_dict2 = parse_cnts_pcts2(housetrained, 'House Trained')
    
    dog_friendly = get_cnts_pcts(df['environment.dogs'])
    dog_friendly_dict1 = parse_cnts_pcts1(dog_friendly, 'Dog Friendly')
    dog_friendly_dict2 = parse_cnts_pcts2(dog_friendly, 'Dog Friendly')
    
    out.update(breeds_dict1)
    out.update(age_dict1)
    out.update(color_dict1)
    out.update(children_dict1)
    out.update(housetrained_dict1)
    out.update(dog_friendly_dict1)
    out.update(breeds_dict2)
    
    out_series = pd.Series(out)
    return(out_series)

In [5]:
def dog_days_gb_func(df, adopt_date=(datetime.datetime.now() - datetime.timedelta(days=1)).date()):
    
    df = df.sort_values('date_saved')
    
    out = {}
    out['Days ad posted'] = (df['date_saved'].max() - df['published_at'].min()).days
    adoptable_days = df[df['status'] == 'adoptable']
    out['Last Adoptable Day'] = adoptable_days['date_saved'].max()
    out['Adopted (last date)'] = df.iloc[-1]['date_saved'] >= adopt_date 
    
    out_series = pd.Series(out)
    last_record = df.iloc[0]
    out_all = pd.concat([out_series, last_record]) 
    return(out_all)

In [6]:
def top_org_gb_func(df):
    out = {}
    out['Count'] = df.shape[0]
    out['Unique dogs'] = df['id'].nunique()
    
    out['Avg Days in Shelter'] = df['Days bw saved and published'].mean().round(1)
    out['Lower 20% days in Shelter'] = np.quantile(df['Days bw saved and published'], 0.2)
    out['Upper 20% days in Shelter'] = np.quantile(df['Days bw saved and published'], 0.8)

    out['Added last week sum'] = df['Added in last week'].sum()
    out['Added last week pct'] = np.round(100* out['Added last week sum']/out['Count'], 2)
    
    out['Added in >4 weeks sum'] = df['Added in >4 weeks'].sum()
    out['Added in >4 weeks pct'] = np.round(100* out['Added in >4 weeks sum']/out['Count'], 2)
    
    out_series = pd.Series(out)
    return(out_series)

In [9]:
class PetFinderEda():
    
    def __init__(self, **kwargs):
        self.db_location = '/home/malcolm/petfinder/data/petfinder.db'
        self.today = datetime.datetime.now() - datetime.timedelta(days=1)
        self.today_str = str(datetime.datetime.now().date())
        self.two_weeks = self.today - datetime.timedelta(days=7)
        self.two_weeks_str = str(self.two_weeks.date())    
        self.debug = True
        self.image_save_folder = f'/home/malcolm/petfinder/data/result_images/{self.today_str}/'
        # Find folder with dog pics 
        folder_base = '/home/malcolm/sym_data_storage/Petfinder/Dogs/'
        folder_loc = datetime.datetime.now().strftime('%Y-%-m')
        self.dogs_pic_folder = folder_base + folder_loc +'/'
        
        
        self.metrics = {}
        self.output_dfs = {}
        self.__dict__.update(**kwargs)
        
        os.makedirs(self.image_save_folder, exist_ok=True)
        pass
    
    def create_con(self):
        self.con = sqlite3.connect(self.db_location)
        self.cursor = self.con.cursor()
        self.cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        logger.info(self.cursor.fetchall())
        
    def get_new_old_existing_dogs(self):

        sql2 = f"""

        with two_weeks_dogs as (
        select id as id_old
        , status as status_old
        , status_changed_at as status_change_old
        , date_saved as date_saved_old
        from dog_10475_30mi
        where date_saved = '{self.two_weeks_str}'
        ) , 
        today_dogs as (
        select id as id_new
        , status as status_new
        , status_changed_at as status_change_new
        , date_saved as date_saved_new
        from dog_10475_30mi
        where date_saved = '{self.today_str}'
        )


        select count(*)
        , count(distinct id_new) as unique_new_dogs
        , count(distinct id_old) as unique_old_dogs
        , sum(case when id_old is null then 1 else 0 end) as new_dogs -- Old doesn't have key so dog must be new
        , sum(case when id_new is null then 1 else 0 end) as old_dogs -- New doesn't have key so dog must be old 
        , sum(case when id_old is not null and id_new is not null then 1 else 0 end) as still_there_dogs
        , min(date_saved_old) as start_date
        , max(date_saved_new) as end_date

        from (
        SELECT id_new, status_new, status_change_new, date_saved_new
        , id_old, status_old, status_change_old, date_saved_old
        from two_weeks_dogs
        left join today_dogs
        on two_weeks_dogs.id_old = today_dogs.id_new

        UNION 

        SELECT id_new, status_new, status_change_new, date_saved_new
        , id_old, status_old, status_change_old, date_saved_old
        from today_dogs 
        left join two_weeks_dogs
        on today_dogs.id_new = two_weeks_dogs.id_old
        )x

        """

        self.new_old_existing = pd.read_sql(sql2, self.con)
        self.output_dfs['New Old Existing'] = self.new_old_existing
        
    def load_raw_data(self):
        self.data_raw = pd.read_sql(f"""
        select dogs.*, orgs.name as org_name 
        from dog_10475_30mi dogs
        left join Organizations orgs
        on dogs.organization_id = orgs.id
        where date_saved between '{self.two_weeks_str}' and '{self.today_str}'
        """ , self.con)
        self.data_raw['published_at'] = pd.to_datetime(self.data_raw['published_at'])
        self.data_raw['status_changed_at'] = pd.to_datetime(self.data_raw['status_changed_at'])
        
        self.data_raw['date_saved'] = pd.to_datetime(self.data_raw['date_saved'], utc=True)
        self.latest_day_df = self.data_raw[self.data_raw['date_saved'] == self.data_raw['date_saved'].max()]
        
        logger.info("Number of datapoints in data_raw: " + str(self.data_raw.shape[0]))
        logger.info("Number of data points in latest day: " + str(self.latest_day_df.shape[0]))
        
    def get_top_orgs(self):
        org_value_counts = get_cnts_pcts(self.latest_day_df['org_name'])
        self.top_n_dog_orgs = org_value_counts.sort_values('Count', ascending=False).iloc[:4].index.tolist()
        logger.info("Top n dog orgs: " + str(self.top_n_dog_orgs))
        self.output_dfs['Top Orgs'] = org_value_counts.iloc[:4]
             
    def get_dog_days_gb(self):
        self.dog_gb = self.data_raw.groupby('id').apply(dog_days_gb_func)
        self.adopted_df = self.dog_gb[self.dog_gb['Adopted (last date)'] == False]
        logger.info("Dog days in shelter df shape: " + str(self.dog_gb.shape))
        # Analysis 
        long_ad_posted_dogs = self.dog_gb[self.dog_gb['Days ad posted'] >= 180]
        long_ad_posted_age_cnts = get_cnts_pcts(long_ad_posted_dogs['age'])\
            .rename({'Count': '6 Mo+ Ad Count', 
                    'Percent': '6 Mo+ Ad Pct'}, axis=1 )
        age_cnts = get_cnts_pcts(self.dog_gb['age'])\
            .rename({'Count': 'Full Pop Count',
                      'Percent': 'Full Pop Pct'}, axis= 1)
        long_ad_posted_age_cnts = pd.merge(long_ad_posted_age_cnts, age_cnts, left_index=True, right_index=True)
        long_ad_posted_age_cnts['Sample Ratio'] = (long_ad_posted_age_cnts['6 Mo+ Ad Pct']/age_cnts['Full Pop Pct']).round(2)
        self.long_ad_posted_age_cnts = long_ad_posted_age_cnts
        self.output_dfs['Long Ad Posted DF'] = self.long_ad_posted_age_cnts
        
    def summarize_latest_day(self):
        all_summaries = {}
        all_summaries['Latest Day'] = summarize_dog_pop(self.latest_day_df)
        all_summaries['Adopted'] = summarize_dog_pop(self.adopted_df)
        for org in self.top_n_dog_orgs:
            all_summaries[org] = summarize_dog_pop(self.latest_day_df[self.latest_day_df['org_name'] == org])
        self.all_summaries_df = pd.concat(all_summaries, axis=1)
        self.output_dfs['All Summaries'] = self.all_summaries_df

    def create_top_orgs_over_time_dfs(self):
        top_orgs = self.output_dfs['Top Orgs'].index.tolist()
        ns_raw = self.data_raw[self.data_raw['org_name'].isin(top_orgs)]
        ns_raw['Days bw saved and published'] = (ns_raw['date_saved'] - ns_raw['published_at']).dt.days
        ns_raw['Added in last week'] = ns_raw['Days bw saved and published'] <= 7
        ns_raw['Added in >4 weeks']  = ns_raw['Days bw saved and published'] >= 28
        self.top_orgs_raw = ns_raw
        self.top_orgs_gb_df = self.top_orgs_raw.groupby(['date_saved', 'org_name']).apply(top_org_gb_func)
        logger.debug("Finished Top Orgs over Time")
        
    def create_top_org_plots(self):
        # Shuffle the dataframe  
        cols = ['Avg Days in Shelter', 'Upper 20% days in Shelter']
        days_in_regroup = pd.concat({x : self.top_orgs_gb_df[x] for x in cols})\
            .reset_index()\
            .rename({'level_0':'Days Measure', 
                    0 : "# of Days"}, axis=1)
        
        # Create ands save first plot 
        n_days_per_org_img = px.line(days_in_regroup, 'date_saved', '# of Days'
            , color='org_name'
            , line_dash='Days Measure'
            , title='Number of Days (average, top decile) in Shelter over Time'                         
            )
        n_days_per_org_img.write_image(self.image_save_folder + f'N Days per Dog.png')
        
        # Create and save second plot 
        n_dogs_in_shelter_img = px.line(self.top_orgs_gb_df.reset_index(), 'date_saved', 'Unique dogs'
           , color = 'org_name'
           , title= 'Number of Dogs in Shelter')
        n_dogs_in_shelter_img.write_image(self.image_save_folder + 'N Dogs in Shelters.png')
        
        # Create and save third plot 
        last_week_dogs_img = px.line(self.top_orgs_gb_df.reset_index(), 'date_saved', 'Added last week pct'
           , color = 'org_name'
           , title= 'Number of Dogs in Shelter <7 days'
           )
        last_week_dogs_img.write_image(self.image_save_folder + 'Added Last Week Dogs.png')
      
        # Create and save third plot 
        dogs_still_there_img = px.line(self.top_orgs_gb_df.reset_index(), 'date_saved', 'Added in >4 weeks pct'
           , color = 'org_name'
           , title= 'Number of Dogs in Shelter >4 weeks'
           )
        dogs_still_there_img.write_image(self.image_save_folder + 'Still There Dogs.png')
        logger.debug('Finished Creating Images')
        
    def get_pics_to_send(self):
        # 4 pictures of dogs to send 
        total_dog_pics = os.listdir(self.dogs_pic_folder)
        pics_to_send = np.random.choice([self.dogs_pic_folder + x for x in total_dog_pics], size = 4)
    
        # Charts 
        charts = os.listdir(self.image_save_folder)
        self.image_paths_to_send = [self.image_save_folder + x for x in charts]
        self.image_paths_to_send.extend(pics_to_send.tolist())
        
    def cleanup(self):
        self.con.commit()
        self.con.close()
        logger.debug('Finished Cleanup ')
        
    def execute(self):
        self.create_con()
        self.get_new_old_existing_dogs()
        self.load_raw_data()
        self.get_top_orgs()
        self.get_dog_days_gb()
        self.summarize_latest_day()
        self.create_top_orgs_over_time_dfs()
        self.create_top_org_plots()
        self.get_pics_to_send()
        self.cleanup()
        

In [10]:
pf_eda = PetFinderEda()
pf_eda.execute()

2021-11-21 14:51:22,129 :: INFO :: [('dog_10475_30mi',), ('cat_10475_30mi',), ('Organizations',), ('Dog_Image_Status',)]
2021-11-21 14:52:02,798 :: INFO :: Number of datapoints in data_raw: 19171
2021-11-21 14:52:02,805 :: INFO :: Number of data points in latest day: 2375
2021-11-21 14:52:02,822 :: INFO :: Top n dog orgs: ['Rescue Dogs Rock NYC', 'Second Chance Rescue', 'Ruff House Rescue', 'Town of Hempstead Animal Shelter']
2021-11-21 14:52:30,715 :: INFO :: Dog days in shelter df shape: (2900, 54)


In [11]:
pf_eda.output_dfs.keys()

dict_keys(['New Old Existing', 'Top Orgs', 'Long Ad Posted DF', 'All Summaries'])

In [12]:
pf_eda.output_dfs['Long Ad Posted DF']

,6 Mo+ Ad Count,6 Mo+ Ad Pct,Full Pop Count,Full Pop Pct,Sample Ratio
Adult,383,62.276,1169,40.310,1.54
Young,140,22.764,748,25.793,0.88
Senior,74,12.033,246,8.483,1.42
Baby,18,2.927,737,25.414,0.12


In [13]:
pf_eda.output_dfs['New Old Existing']

,count(*),unique_new_dogs,unique_old_dogs,new_dogs,old_dogs,still_there_dogs,start_date,end_date
0,2820,2375,2459,361,445,2014,2021-11-13,2021-11-21


In [14]:
pf_eda.output_dfs['Top Orgs']

,Count,Percent
Rescue Dogs Rock NYC,219,10.153
Second Chance Rescue,101,4.682
Ruff House Rescue,87,4.033
Town of Hempstead Animal Shelter,55,2.550


In [15]:
pf_eda.output_dfs['All Summaries']

,Latest Day,Adopted,Rescue Dogs Rock NYC,Second Chance Rescue,Ruff House Rescue,Town of Hempstead Animal Shelter
Count,2375,448,219,101,87,55
Top Breed Name,Labrador Retriever,Labrador Retriever,Labrador Retriever,American Staffordshire Terrier,Chihuahua,Terrier
Top Breed %,12.842,18.527,21.461,37.624,22.989,61.818
Top Age Name,Adult,Baby,Adult,Adult,Young,Adult
Top Age %,43.705,47.321,49.315,50.495,51.724,85.455
Top Color Name,Black,Black,None,"Tricolor (Brown, Black, & White)",Black,None
Top Color %,21.649,23.125,None,50,34.483,None
Top Children Name,1,1,1,1,0,0
Top Children %,78.215,95.279,98.122,83.333,100,100
Top House Trained Name,0,0,0,0,0,0


In [16]:
html_dict = {k:v.to_html() for k, v in pf_eda.output_dfs.items() }
body_html = f"""
This email contains a report of the <b>dogs</b> listed on Petfinder.com from {pf_eda.two_weeks_str} to {pf_eda.today_str} 
for all locations 30 miles from 10475. 

It contains summary information about the Number of dogs in shelters as of {pf_eda.today_str}, top organizations
, dogs who have been in shelters the longest. 
<br>
<br>
<b>Adoptions</b>
{html_dict['New Old Existing']}
<br>
<br>
<b>Time in Shelter </b>
{html_dict['Long Ad Posted DF']}
<br>
<br>
<b>Shelters with Most Dogs </b>
{html_dict['Top Orgs']}
<br>
<br>
<b>Summary of Dogs in Shelter </b>
{html_dict['All Summaries']}


"""


In [17]:
charts = os.listdir(pf_eda.image_save_folder)
full_path_charts = [pf_eda.image_save_folder + x for x in charts]

In [18]:
folder_base = '/home/malcolm/sym_data_storage/Petfinder/Dogs/'
folder_loc = datetime.datetime.now().strftime('%Y-%-m')
folder_path = folder_base + folder_loc +'/'
total_dog_pics = os.listdir(folder_path)
pics_to_send = np.random.choice([folder_path + x for x in total_dog_pics], size = 4)

In [ ]:
full_path_charts.extend(pics_to_send.tolist())

## Email

In [19]:
sys.path.append('/home/malcolm/EmailSender/')

In [20]:
from EmailSender import EmailSender
from IPython.core.display import display, HTML, Image

In [21]:
message_params = {}
message_params['Subject'] = f"Petfinder Dogs from {pf_eda.two_weeks_str} to {pf_eda.today_str}"
message_params['Body'] = body_html
message_params['Image_paths'] = pf_eda.image_paths_to_send

In [22]:
email_sender = EmailSender(**message_params)
email_sender.execute()

In [ ]:
[(x, type(getattr(pf_eda, x))) for x in dir(pf_eda) if x[:2] != '__' ]

In [ ]:
Image(pf_eda.image_save_folder+'/Still There Dogs.png')

## Begin OLD STUFF

In [ ]:
con = sqlite3.connect('/home/malcolm/petfinder/data/petfinder.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

In [ ]:
today = datetime.datetime.now()
today_str = str(datetime.datetime.now().date())
two_weeks = today - datetime.timedelta(days=7)
two_weeks_str = str(two_weeks.date())

In [ ]:
sql2 = f"""

with two_weeks_dogs as (
select id as id_old
, status as status_old
, status_changed_at as status_change_old
, date_saved as date_saved_old
from dog_10475_30mi
where date_saved = '{two_weeks_str}'
) , 
today_dogs as (
select id as id_new
, status as status_new
, status_changed_at as status_change_new
, date_saved as date_saved_new
from dog_10475_30mi
where date_saved = '{today_str}'
)


select count(*) as total_dogs
, count(distinct id_new) as unique_new_dogs
, count(distinct id_old) as unique_old_dogs
, sum(case when id_old is null then 1 else 0 end) as new_dogs -- Old doesn't have key so dog must be new
, sum(case when id_new is null then 1 else 0 end) as old_dogs -- New doesn't have key so dog must be old 
, sum(case when id_old is not null and id_new is not null then 1 else 0 end) as still_there_dogs
, min(date_saved_old) as start_date
, max(date_saved_new) as end_date

from (
SELECT id_new, status_new, status_change_new, date_saved_new
, id_old, status_old, status_change_old, date_saved_old
from two_weeks_dogs
left join today_dogs
on two_weeks_dogs.id_old = today_dogs.id_new

UNION 

SELECT id_new, status_new, status_change_new, date_saved_new
, id_old, status_old, status_change_old, date_saved_old
from today_dogs 
left join two_weeks_dogs
on today_dogs.id_new = two_weeks_dogs.id_old
)x

"""

demo1 = pd.read_sql(sql2, con)
demo1

In [ ]:
data_raw = pd.read_sql(f"""
select dogs.*, orgs.name as org_name 
from dog_10475_30mi dogs
left join Organizations orgs
on dogs.organization_id = orgs.id
where date_saved between '{two_weeks_str}' and '{today_str}'
""" , con)
data_raw['published_at'] = pd.to_datetime(data_raw['published_at'])
data_raw['status_changed_at'] = pd.to_datetime(data_raw['status_changed_at'])
data_raw['date_saved'] = pd.to_datetime(data_raw['date_saved'], utc=True)

data = data_raw[data_raw['date_saved'] == data_raw['date_saved'].max()]
print(data.shape)
data.head()

In [ ]:
data_raw['id'].nunique()

In [ ]:
data_raw.shape

## Peak at some attributes

In [ ]:
data.columns

In [ ]:
data[data['status_changed_at'] <= '2021-01-01'].shape
# Probably data quality issue.. should investigate or drop these records

In [ ]:
status_change = data[data['status_changed_at'] != data['published_at']]
status_change.shape

In [ ]:
status_change

In [ ]:
status_change.status.value_counts()

In [ ]:
summary = summarize_dog_pop(data)
summary

In [ ]:
data[data['org_name'] == 'Rescue Dogs Rock NYC']

In [ ]:
summary1 = summarize_dog_pop(data[data['org_name'] == 'Rescue Dogs Rock NYC'])
summary1

In [ ]:
summary2 = summarize_dog_pop(data[data['org_name'] == 'Second Chance Rescue'])
summary2

In [ ]:
org_value_counts = get_cnts_pcts(data['org_name'])
org_value_counts

In [ ]:
# # of shelter with more than 10 dogs 
org_value_counts[org_value_counts['Count'] >= 10].shape[0]

In [ ]:
org_value_counts['Count'].hist()

In [ ]:
data.iloc[1]

## Dog Status Change

In [ ]:
dog_gb = data_raw.groupby('id').apply(dog_days_gb_func)
print("Shape: ", dog_gb.shape)
dog_gb

In [ ]:
(dog_gb['Adopted (last date)'] == False).sum()

In [ ]:
adopted = dog_gb[dog_gb['Adopted (last date)'] == False]
summarize_dog_pop(adopted)

In [ ]:
adopted['Days ad posted'].describe()

In [ ]:
dog_gb['Days ad posted'].describe()

In [ ]:
long_ad_posted_dogs = dog_gb[dog_gb['Days ad posted'] >= 180]
long_ad_posted_dogs.shape

In [ ]:
long_ad_posted_dogs['age'].value_counts()

In [ ]:

age_cnts

In [ ]:
long_ad_posted_age_cnts = get_cnts_pcts(long_ad_posted_dogs['age'])\
    .rename({'Count': '6 Mo+ Ad Count', 
            'Percent': '6 Mo+ Ad Pct'}, axis=1 )
age_cnts = get_cnts_pcts(dog_gb['age'])\
    .rename({'Count': 'Full Pop Count',
              'Percent': 'Full Pop Pct'}, axis= 1)
long_ad_posted_age_cnts = pd.merge(long_ad_posted_age_cnts, age_cnts, left_index=True, right_index=True)
long_ad_posted_age_cnts['Sample Ratio'] = (long_ad_posted_age_cnts['6 Mo+ Ad Pct']/age_cnts['Full Pop Pct']).round(2)
long_ad_posted_age_cnts

In [ ]:
age_cnts

## Top Places over time


In [ ]:
import plotly.express as px
import numpy as np

In [ ]:
data_raw.shape

In [ ]:
org_value_counts.sort_values('Count', ascending=False).iloc[:4]

In [ ]:
top_n_dog_orgs = org_value_counts.sort_values('Count', ascending=False).iloc[:4].index.tolist()
top_n_dog_orgs
# str(top_n_dog_orgs)[1:-1]

In [ ]:
ns_raw = data_raw[data_raw['org_name'].isin(top_n_dog_orgs)]
print("Shape: ", ns_raw.shape)
ns_raw.head()

In [ ]:
ns_raw.columns

In [ ]:
ns_raw['Days bw saved and published'] = (ns_raw['date_saved'] - ns_raw['published_at']).dt.days
ns_raw['Added in last week'] = ns_raw['Days bw saved and published'] <= 7
ns_raw['Added in <4 weeks']  = ns_raw['Days bw saved and published'] >= 28



In [ ]:
def gb_func(df):
    out = {}
    out['Count'] = df.shape[0]
    out['Unique dogs'] = df['id'].nunique()
    
    out['Avg Days in Shelter'] = df['Days bw saved and published'].mean().round(1)
    out['Lower 20% days in Shelter'] = np.quantile(df['Days bw saved and published'], 0.2)
    out['Upper 20% days in Shelter'] = np.quantile(df['Days bw saved and published'], 0.8)

    out['Added last week sum'] = df['Added in last week'].sum()
    out['Added last week pct'] = np.round(100* out['Added last week sum']/out['Count'], 2)
    
    out['Added in <4 weeks sum'] = df['Added in <4 weeks'].sum()
    out['Added in <4 weeks pct'] = np.round(100* out['Added in <4 weeks sum']/out['Count'], 2)
    
    

    
    out_series = pd.Series(out)
    return(out_series)

In [ ]:
ns_gb = ns_raw.groupby(['date_saved', 'org_name']).apply(gb_func)
ns_gb

In [ ]:
cols = ['Avg Days in Shelter', 'Upper 20% days in Shelter']
days_in_regroup = pd.concat({x:ns_gb[x] for x in cols})\
    .reset_index()\
    .rename({'level_0':'Days Measure', 
            0 : "# of Days"}, axis=1)
days_in_regroup.head()

In [ ]:
px.line(days_in_regroup, 'date_saved', '# of Days'
        , color='org_name'
        , line_dash='Days Measure'
        , )

In [ ]:
px.line(ns_gb.reset_index(), 'date_saved', 'Unique dogs'
       , color = 'org_name'
       , title= 'Number of Dogs in Shelter')

In [ ]:
px.line(ns_gb.reset_index(), 'date_saved', 'Added last week pct'
       , color = 'org_name'
#        , title= 'Number of Dogs in Shelter'
       )

In [ ]:
px.line(ns_gb.reset_index(), 'date_saved', 'Added in <4 weeks pct'
       , color = 'org_name'
#        , title= 'Number of Dogs in Shelter'
       )